In [1]:
import pandas as pd
import numpy as np
import ast
import textwrap

from copy import deepcopy
import re
import json

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import spacy
from bertopic import BERTopic

#Representación de topics
from bertopic.vectorizers import ClassTfidfTransformer
#Embedding model
from sentence_transformers import SentenceTransformer
#UMAP para reducir la dimensionalidad
from umap import UMAP
#clusterizador
from hdbscan import HDBSCAN
#Mapa de texto a vector 
from sklearn.feature_extraction.text import CountVectorizer
#representacion fine-tuned de bert topic: https://maartengr.github.io/BERTopic/index.html#fine-tune-topic-representations
from bertopic.representation import KeyBERTInspired

/home/alejandro/.virtualenvs/mainenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-08 22:28:56.314316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746761337.452807   60005 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746761337.660573   60005 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746761339.275423   60005 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more th

# Objetivo

Se tienen unas vacantes laborales para el sector energético. Cada vacante tienen un listado de conocimientos. Se quiere agrupar los conocimientos en categorias estandar para discernir cuales son los conocimientos que está demandando el mercado laboral.

## Referencias

https://maartengr.github.io/BERTopic/index.html#variations

## Funciones

In [2]:
def safe_parse_conocimientos(value):
    '''
    Cambia de formato las entradas de la forma
    "'palabra_1','palabra_2',..., 'palabra_n'"
    a
    ['palabra_1','palabra_2,...,'palabra_n']
    '''
    try:
        # Si no está entre corchetes, lo envolvemos
        if not value.strip().startswith("["):
            value = f"[{value}]"
        return ast.literal_eval(value)
    except Exception:
        return [str(value)]  # Si no se puede evaluar, lo devolvemos como lista con un solo string

## Leer datos

In [3]:
conoc_df=pd.read_excel("../data/conoc_tej.xlsx",index_col=False)

In [4]:
print(f"Numero de filas: {conoc_df.shape[0]} \nNumero de columnas: {conoc_df.shape[1]}")
conoc_df.head()

Numero de filas: 3087 
Numero de columnas: 26


,id,titulo_vacante,descripcion,tipo_documento,cod_depto,fecha_inicio,fecha_cierre,registro_mes,registro_ano,tipo1prestador,...,depto,mpio,rangos_salariales,numero_documento_anom,codigo,descripcion_input,profesion,educacion,conocimientos,clasif_cine_nombre
0,180,auxiliar de servicio tecnico industrial,importante multinacional del sector energetico...,NI,11,44942,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.500.001 - $2.000.000,fdd3b5bff414026a8d0b7c09bc232281d192797cec6b2b...,OFERTA_181,auxiliar de servicio tecnico industrial; impor...,'Auxiliar servicio técnico industrial',"'Mecánico', 'Electromecánico'","'mantenimiento de maquinaria industrial', 'man...",71.0-Ingeniería y profesiones afines
1,368,tecnico fumigador - con moto,importante empresa dedicada a la prestacion de...,NI,11,44945,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.000.001 - $1.500.000,fdd3b5bff414026a8d0b7c09bc232281d192797cec6b2b...,OFERTA_369,tecnico fumigador - con moto; importante empre...,'Técnico fumigador','Bachiller',"'Control integrado de plagas', 'Control de roe...",1.0-Programas y certificaciones básicos
2,637,ingeniero preventa,importante empresa requiere para su equipo de ...,NI,11,44950,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$2.000.001 - $3.000.000,f1bb3499cfe1370bbf73106827e6ba0c3b6fe5c06db679...,OFERTA_638,ingeniero preventa; importante empresa requier...,'Ingeniero preventa',"'Ingeniería electrónica', 'Mecatrónica', 'Tele...","'Proyectos de seguridad electrónica', 'Dimensi...",41.0-Educación comercial y administración
3,702,tecnico electricista - lugar de trabajo: bogota,importante empresa requiere tecnico electricis...,NI,11,44936,30/01/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.000.001 - $1.500.000,53547f1b0955425b627d74c9865433f6ef9137a8ecb8c0...,OFERTA_703,tecnico electricista - lugar de trabajo: bogot...,'Técnico electricista','NO_APLICA','Mantenimiento preventivo y correctivo de rede...,1.0-Programas y certificaciones básicos
4,1123,vendedor/a - experiencia en ventes telefonicas,importante compania del sector de saneamiento ...,NI,11,44942,17/03/2023,1,2023,Privado,...,"BOGOTA, D. C.",BOGOTA D.C.,$1.000.001 - $1.500.000,e1c11017d0aff28b84b75cef8b7a4d5ae5e5f140530b9c...,OFERTA_1124,vendedor/a - experiencia en ventes telefonicas...,'Vendedor','NO_APLICA',"'Ventas telefónicas', 'Call center'",1.0-Programas y certificaciones básicos


In [5]:
#descripcion de la primera vacante
wrapped_text = textwrap.fill(conoc_df['descripcion'][0], width=80)
print(wrapped_text)

importante multinacional del sector energetico requiere para su equipo de
trabajo tecnico o tecnologo mecanico, electromecanico o a fines, con minimo tres
anos de experiencia en el desarrollo de trabajos asociados con el mantenimiento
de maquinaria industrial (no locativo), manejo de bombas de vacio, equipos de
soldadura, mecanica hidraulica. te ofrecemosii salario:2.000.000 + prestaciones
de ley horario de lunes a sabado - disponibilidad domingo o festivos lugar de
trabajo tenjo cundinamarca contrato obra o labor 12 meses beneficio de casino y
ruta


In [6]:
#Conocimientos de la vacante
conoc_df['conocimientos'][0]

"'mantenimiento de maquinaria industrial', 'manejo de bombas de vacío', 'equipos de soldadura', 'mecánica hidráulica'"

## Modificacion Datos

Existen entradas con valor no validos

In [7]:
#Reemplar valores no validos por np.nan
conoc_df=conoc_df.replace('NA',np.nan)
conoc_df=conoc_df.replace("'NO_APLICA'",np.nan)

In [8]:
#eliminar las entradas con nans en la columna conocimientos
conoc_df = conoc_df.dropna(subset=['conocimientos'])

In [9]:
#Creamos una columna con la info de la vacante haciendo enfasis en los conocimientos
conoc_df['descr_vacante_ext'] = conoc_df.apply(
    lambda row: np.nan if pd.isna(row['conocimientos']) else
    "Se requieren los siguientes conocimientos: " +
    ", ".join([f"{i+1}. {item.strip()}" for i, item in enumerate(re.findall(r"'(.*?)'", row['conocimientos']))]) +
    f", para la siguiente vacante laboral: {row['descripcion']}",
    axis=1
)

## Corpus
Se generan 3 corpus
>Corpus individual: Cada documento es un conocimiento

>Corpus extendido: Cada documento es la especificacion de los conocimientos junto con la vacante

>Corpus original: Cada documento es de la forma "Los conocimientos para la vacante laboral son: " + listado de conocimientos

In [10]:
#Generamos una lista con los conocimientos individuales
#En este corpus cada documento es un conocimiento especifico
#se eliminan conocimientos que literalmente son duplicados
corpus_individual = list(dict.fromkeys(
    conoc_df['conocimientos'].apply(safe_parse_conocimientos).sum()
))
print(f"Numero de documentos en el corpus_individual:{len(corpus_individual)}")

#En este corpus cada documento son los conocimientos y la descripcion de la vacante
corpus_extendido = conoc_df['descr_vacante_ext']
print(f"Numero de documentos en el corpus_extendido:{len(corpus_extendido)}")

#En este corpus cada documento es un listado de conocimientos
corpus_original = conoc_df['conocimientos'].apply(
    lambda x: np.nan if pd.isna(x) else
    "Los conocimientos para la vacante laboral son: " +
    ", ".join([f"{i+1}. {item.strip()}" for i, item in enumerate(re.findall(r"'(.*?)'", x))])
)
print(f"Numero de documentos en el corpus_originall:{len(corpus_original)}")

Numero de documentos en el corpus_individual:8671
Numero de documentos en el corpus_extendido:3035
Numero de documentos en el corpus_originall:3035


### Ejemplo de los corpus

In [11]:
corpus_individual[3082]

'Digitalizacion'

In [12]:
corpus_extendido[3082]

'Se requieren los siguientes conocimientos: 1. Diseño, implementación, mantenimiento y soporte a redes eléctricas cableadas, 2. Redes de cableado estructurado, para la siguiente vacante laboral: importante empresa del sector tecnologico requiere para su equipo de trabajo ingeniero electronico nivel 3 requisitos:* profesional en ingenieria  electrnica o elctrica o telemtica* certificado de trabajo en alturas.* contar con targeta profesional *  mnimo cuatro (4) aos de experiencia profesional certificada, contabilizada a partir de la fecha de emisin de la tarjeta profesional* mnimo  dos(2) aos de experiencia profesional especfica certificada en diseo de implementacion, mantenimiento y soporte a redes electricas cableadas y a redes de cableado estructurado.beneficios:salario:  $ 3.392.194contrato: obra labor horario: lunes a viernes 7:00am - 5:00pm si cumples con los requisitos y te encuentras interesado, postulate revisaremos tu hoja de vida y nos pondremos en contactote esperamosi'

In [13]:
corpus_original[3082]

'Los conocimientos para la vacante laboral son: 1. Diseño, implementación, mantenimiento y soporte a redes eléctricas cableadas, 2. Redes de cableado estructurado'

## Instanciar modelo

In [14]:
#spamsih stop words
nlp = spacy.blank("es")
spanish_stopwords = nlp.Defaults.stop_words
del(nlp)

c-TF-IDF will show you what words are most important for each topic, relative to the other topics in the corpus_individual. This allows you to identify topic-specific terms that help characterize the content of each topic.

### Modelo 1
Incluye c-TF-IDF

In [15]:
#Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
#UMAP para reducir la dimensionalidad de los embeddings
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=123)
#Clusterizador de emebeddings
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
#Tokenizador de topicos
vectorizer_model = CountVectorizer(stop_words=list(spanish_stopwords), min_df=2, ngram_range=(1,3))
#Create topic representation
ctfidf_model = ClassTfidfTransformer()
#representacion fine-tuned de bert topic:
representation_model = KeyBERTInspired()


model_1 = BERTopic(
    embedding_model=embedding_model,            # Step 1 - Extract embeddings
    umap_model=umap_model,                      # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,          # Step 4 - Tokenize topics_individual
    ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words
    representation_model=representation_model,  # Step 6 - (Optional) Fine-tune topic representations
    language= "spanish"
    )#"spanish"#"multilingual

### Modelo 2
Excluye el fine-tuning (representation_model)

In [16]:
#Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
#UMAP para reducir la dimensionalidad de los embeddings
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=123)
#Clusterizador de emebeddings
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
#Tokenizador de topicos
vectorizer_model = CountVectorizer(stop_words=list(spanish_stopwords), min_df=2, ngram_range=(1,3))
#Create topic representation
ctfidf_model = ClassTfidfTransformer()

model_2 = BERTopic(
    embedding_model=embedding_model,            # Step 1 - Extract embeddings
    umap_model=umap_model,                      # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,          # Step 4 - Tokenize topics_individual
    ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words    
    language= "spanish"
    )#"spanish"#"multilingual

### Modelo 3
- El embedding model es de JINA
- Excluye el fine-tuning (representation_model)

Por el momento mi computador no lograr correr el modelo

In [17]:
from transformers.pipelines import pipeline
from sentence_transformers import SentenceTransformer

In [18]:
'''
#Extract embeddings
embedding_model = SentenceTransformer("jinaai/jina-embeddings-v2-base-es", trust_remote_code=True)
#UMAP para reducir la dimensionalidad de los embeddings
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=123)
#Clusterizador de emebeddings
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
#Tokenizador de topicos
vectorizer_model = CountVectorizer(stop_words=list(spanish_stopwords), min_df=2, ngram_range=(1,3))
#Create topic representation
ctfidf_model = ClassTfidfTransformer()

model_2 = BERTopic(
    embedding_model=embedding_model,            # Step 1 - Extract embeddings
    umap_model=umap_model,                      # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,          # Step 4 - Tokenize topics_individual
    ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words    
    language= "spanish"
    )#"spanish"#"multilingual
'''

'\n#Extract embeddings\nembedding_model = SentenceTransformer("jinaai/jina-embeddings-v2-base-es", trust_remote_code=True)\n#UMAP para reducir la dimensionalidad de los embeddings\numap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric=\'cosine\', random_state=123)\n#Clusterizador de emebeddings\nhdbscan_model = HDBSCAN(min_cluster_size=50, metric=\'euclidean\', cluster_selection_method=\'eom\', prediction_data=True)\n#Tokenizador de topicos\nvectorizer_model = CountVectorizer(stop_words=list(spanish_stopwords), min_df=2, ngram_range=(1,3))\n#Create topic representation\nctfidf_model = ClassTfidfTransformer()\n\nmodel_2 = BERTopic(\n    embedding_model=embedding_model,            # Step 1 - Extract embeddings\n    umap_model=umap_model,                      # Step 2 - Reduce dimensionality\n    hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings\n    vectorizer_model=vectorizer_model,          # Step 4 - Tokenize topics_individual\n    ctfi

### Modelo 4
- El embedding model es distiluse-base-multilingual-cased-v1
- Excluye el fine-tuning (representation_model)

In [17]:
#Extract embeddings
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
#UMAP para reducir la dimensionalidad de los embeddings
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=123)
#Clusterizador de emebeddings
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
#Tokenizador de topicos
vectorizer_model = CountVectorizer(stop_words=list(spanish_stopwords), min_df=2, ngram_range=(1,3))
#Create topic representation
ctfidf_model = ClassTfidfTransformer()

model_2_multilingual = BERTopic(
    embedding_model=embedding_model,            # Step 1 - Extract embeddings
    umap_model=umap_model,                      # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,          # Step 4 - Tokenize topics_individual
    ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words    
    language= "spanish"
    )#"spanish"#"multilingual

## Resultados para cada uno de los corpus

- corpus individual
- corpus extendido
- corpus original 

In [18]:
# Corpus asociados a cada modelo
corpus_map = {
    "individual": corpus_individual,
    "individual_multilingual": corpus_individual,
    "extendido": corpus_extendido,
    "original": corpus_original
}

In [19]:
model_map = {
    "individual": deepcopy(model_2),
    "individual_multilingual": deepcopy(model_2_multilingual),
    "extendido": deepcopy(model_1),
    "original": deepcopy(model_1),
}

In [20]:
# Diccionarios para guardar resultados
topics_map = {}
probs_map = {}

# Aplicar fit_transform y guardar resultados
for name in ["individual","individual_multilingual", "extendido", "original"]:
    topics, probs = model_map[name].fit_transform(corpus_map[name])
    topics_map[name] = topics
    probs_map[name] = probs
    # Si quieres asegurar que no se modifican después
    model_map[name] = deepcopy(model_map[name])

In [21]:
for name, model in model_map.items():
    print(f"\n{'*' * 30} Resumen para {name} {'*' * 30}")
    print(f"Número de tópicos: {len(model.get_topics())}")
    print("\nResumen de la cardinalidad:")
    display(model.get_topic_freq()[['Count']].describe())


****************************** Resumen para individual ******************************
Número de tópicos: 48

Resumen de la cardinalidad:


,Count
count,48.000000
mean,180.645833
std,404.537008
min,51.000000
25%,65.000000
50%,89.500000
75%,140.750000
max,2837.000000



****************************** Resumen para individual_multilingual ******************************
Número de tópicos: 48

Resumen de la cardinalidad:


,Count
count,48.000000
mean,180.645833
std,447.587647
min,53.000000
25%,75.500000
50%,97.500000
75%,136.250000
max,3165.000000



****************************** Resumen para extendido ******************************
Número de tópicos: 3

Resumen de la cardinalidad:


,Count
count,3.000000
mean,1011.666667
std,1639.412801
min,21.000000
25%,65.500000
50%,110.000000
75%,1507.000000
max,2904.000000



****************************** Resumen para original ******************************
Número de tópicos: 6

Resumen de la cardinalidad:


,Count
count,6.000000
mean,505.833333
std,660.844132
min,77.000000
25%,118.000000
50%,163.500000
75%,629.000000
max,1744.000000


# Visualizar

## Modelo con corpus donde cada documento es un unico conocimiento

In [33]:
fig = model_map["individual"].visualize_topics()
fig.write_html("../output/topics_individual.html")

## Modelo multilingual con corpus donde cada documento es un unico conocimiento

In [34]:
fig = model_map["individual_multilingual"].visualize_topics()
fig.write_html("../output/topics_individual_multilingual.html")

## Modelo entrenado con corpus donde cada documento es un unico conocimiento

In [25]:
model_map["individual"].generate_topic_labels()

['-1_tableros_gestión_análisis',
 '0_office_software_programas',
 '1_indicadores_técnico_comunicación',
 '2_eléctricos_eléctrico_eléctrica',
 '3_ambiental_ambientales_gestión ambiental',
 '4_sector_gas_eficiencia',
 '5_proyectos_procesos_producción',
 '6_mantenimiento_plantas_preventivo',
 '7_seguridad_seguridad salud_salud',
 '8_solar_energía_sistemas',
 '9_iso_normas_ntc',
 '10_redes_redes eléctricas_eléctricas',
 '11_trabajo_supervisión_trabajos',
 '12_equipos_mantenimiento equipos_soldadura',
 '13_herramientas_maquinaria_máquinas',
 '14_manejo_manejo integral_manejo redes',
 '15_ventas_venta_mecánica',
 '16_información_informes_reportes',
 '17_certificación_certificaciones_certificado',
 '18_cotizaciones_ecosistemas_económicos',
 '19_clientes_cliente_orientación',
 '20_enfriamiento_inversores_chiller',
 '21_licencia_conducción_c1',
 '22_vehículos_transporte_carga',
 '23_control_plagas_controles',
 '24_iluminación_instalación_instalaciones',
 '25_registro_proveedores_facturas',
 '26

## Modelo multilingual entrenado con corpus donde cada documento es un unico conocimiento

In [26]:
model_map["individual_multilingual"].generate_topic_labels()

['-1_mantenimiento_control_manejo',
 '0_transporte_cambio_cartera',
 '1_tarjeta_conte_clase',
 '2_proyectos_diseño_proyectos proyectos',
 '3_obra_obras_trabajo',
 '4_normas_normatividad_legales',
 '5_fibra_redes_comunicación',
 '6_eléctricas_redes eléctricas_redes',
 '7_energía_energética_eficiente',
 '8_ambientales_ambiental_impactos ambientales',
 '9_clientes_cliente_relaciones',
 '10_análisis_analisis_análisis datos',
 '11_técnico_problemas_técnicas',
 '12_residuos_limpieza_peligrosos',
 '13_gestión_administración_gestion',
 '14_solares_solar_energía',
 '15_informes_reportes_indicadores',
 '16_aire_hvac_refrigeración',
 '17_eléctrico_electrónica_eléctricos',
 '18_certificación_certificaciones_certificado',
 '19_software_sap_programación',
 '20_sistema gestión_sistema_sistemas gestión',
 '21_sistemas_acceso_sistema',
 '22_planes_planos_planificación',
 '23_plantas_césped_poda',
 '24_procesos_procedimientos_proceso',
 '25_herramientas_manejo herramientas_manuales',
 '26_seguridad_segu

# Hierarchical topic modelling

## Modelo entrenado con corpus donde cada documento es un unico conocimiento

In [27]:
hierarchical_individual = model_map["individual"].hierarchical_topics(corpus_map["individual"])

100%|██████████████████████████████████████████| 46/46 [00:00<00:00, 257.99it/s]


In [32]:
fig = model_map["individual"].visualize_hierarchy(hierarchical_topics=hierarchical_individual)
fig.write_html("../output/hierarchy_individual.html")

## Modelo multilingual entrenado con corpus donde cada documento es un unico conocimiento

In [29]:
hierarchical_individual_multilingual = model_map["individual_multilingual"].hierarchical_topics(corpus_map["individual_multilingual"])

100%|██████████████████████████████████████████| 46/46 [00:00<00:00, 346.66it/s]


In [31]:
fig = model_map["individual_multilingual"].visualize_hierarchy(hierarchical_topics=hierarchical_individual_multilingual)
fig.write_html("../output/hierarchy_individual_multilingual.html")